## GET TEST DATA 

In [1]:
# import pandas as pd 
# data_path="/home/user/ksharma/ks_thesis/ma-thesis/pipeline_test/data_files/MASTER_TEST_SET.csv"
# test_set = pd.read_csv(data_path)
# test_set = test_set[test_set['valid'] == 1]
# test_set['expected_skills'] = test_set['expected_text'].str.findall(r'<SoftSkill>(.*?)</SoftSkill>')
# test_set
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge


# TESTING SAVED MODELS

In [2]:
import pandas as pd
import re
pd.set_option("display.max_colwidth", None)

results_bloom = pd.read_csv("/home/user/ksharma/ks_thesis/ma-thesis/bloom_peft_lora/bloom_7bn_results_ORIG_15epochs.csv")
results_bloom = results_bloom[results_bloom['valid'] == 1]
results_bloom = results_bloom.drop(columns='Unnamed: 0', axis = 0)
results_bloom = results_bloom.drop(columns='expected_skills', axis = 0)
results_bloom = results_bloom.drop(columns='generated_skills', axis = 0)

# Function to extract skills between "<SoftSkill_C>" and "</SoftSkill_C>"
def extract_skills_c(text):
    return re.findall(r'<SoftSkill_C>(.*?)<\/SoftSkill_C', text)

# Function to extract skills between "<SoftSkill>"
def extract_skills(text):
    return re.findall(r'<SoftSkill>(.*?)<\/SoftSkill', text)

# Apply the functions to 'input', 'expected_text', and 'generated_text' columns
results_bloom['input_skills_C'] = results_bloom['input'].apply(extract_skills_c)
# df['input_skills'] = df['input'].apply(extract_skills)
results_bloom['input_skills_all'] = results_bloom['input'].str.findall(r'(?:<SoftSkill>|<SoftSkill_C>)(.*?)(?:</SoftSkill|</SoftSkill_C)')
results_bloom['expected_skills_all'] = results_bloom['expected_text'].str.findall(r'<SoftSkill>(.*?)<\/SoftSkill')
results_bloom['generated_skills_all'] = results_bloom['generated_text'].str.findall(r'<SoftSkill>(.*?)<\/SoftSkill')

# Function to find the corresponding skills in 'expected_text' and 'generated_text'
def find_C_expected_skills(row):
    corresponding_skills = []
    for skill_c in row['input_skills_C']:
        if skill_c in row['input_skills_all']:
            idx = row['input_skills_all'].index(skill_c)
            corresponding_skills.append(row['expected_skills_all'][idx])
            # corresponding_skills.append(row['generated_skills_new'][idx])
    return corresponding_skills

def find_C_generated_skills(row):
    corresponding_skills = []
    for skill_c in row['input_skills_C']:
        if skill_c in row['input_skills_all']:
            idx = row['input_skills_all'].index(skill_c)
            corresponding_skills.append(row['generated_skills_all'][idx])
            # corresponding_skills.append(row['generated_skills_new'][idx])
    return corresponding_skills

# Apply the function to create 'corresponding_skills' column
results_bloom['expected_skills_C'] = results_bloom.apply(find_C_expected_skills, axis=1)
results_bloom['generated_skills_C'] = results_bloom.apply(find_C_generated_skills, axis=1)


results_bloom.head(3)


,orig_idx,input,expected_text,generated_text,Problem type,valid,input_skills_C,input_skills_all,expected_skills_all,generated_skills_all,expected_skills_C,generated_skills_C
0,752,"Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill_C>dienstleitungsorientierten</SoftSkill_C>, <SoftSkill_C>verantwortungsbewussten</SoftSkill_C> wie <SoftSkill>teamorientierten handeln</SoftSkill>.","Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill>dienstleitungsorientierten handeln</SoftSkill>, <SoftSkill>verantwortungsbewussten handeln</SoftSkill> wie <SoftSkill>teamorientierten handeln</SoftSkill>.","Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill>dienstleitungsorientierten handeln</SoftSkill>, <SoftSkill>verantwortungsbewussten handeln</SoftSkill> wie <SoftSkill>teamorientierten handeln</SoftSkill>.",MC,1.0,"[dienstleitungsorientierten, verantwortungsbewussten]","[dienstleitungsorientierten, verantwortungsbewussten, teamorientierten handeln]","[dienstleitungsorientierten handeln, verantwortungsbewussten handeln, teamorientierten handeln]","[dienstleitungsorientierten handeln, verantwortungsbewussten handeln, teamorientierten handeln]","[dienstleitungsorientierten handeln, verantwortungsbewussten handeln]","[dienstleitungsorientierten handeln, verantwortungsbewussten handeln]"
1,765,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill_C>hoher Dienstleistungs-</SoftSkill_C> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill>hoher Dienstleistungsbereitschaft</SoftSkill> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill>hoher Dienstleistungsbereitschaft</SoftSkill> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,HOWA,1.0,[hoher Dienstleistungs-],"[Arbeitsweise ist proaktiv, hoher Dienstleistungs-, Einsatzbereitschaft]","[Arbeitsweise ist proaktiv, hoher Dienstleistungsbereitschaft, Einsatzbereitschaft]","[Arbeitsweise ist proaktiv, hoher Dienstleistungsbereitschaft, Einsatzbereitschaft]",[hoher Dienstleistungsbereitschaft],[hoher Dienstleistungsbereitschaft]
2,1655,"Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill_C>psychische</SoftSkill_C> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill_C>Team-</SoftSkill_C> und <SoftSkill>Kommunikations-Fähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.","Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill>psychische Gesundheit</SoftSkill> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill>Teamfähigkeit</SoftSkill> und <SoftSkill>Kommunikationsfähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.","Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill>psychische Gesundheit</SoftSkill> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill>Teamfreudigkeit</SoftSkill> und <SoftSkill>Kommunikations-Fähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.",HMDC,1.0,"[psychische, Team-]","[psychische, physische Gesundheit, Kontaktfreudigkeit, gute Auffassungsgabe, Team-, Kommunikations-Fähigkeit, praktisches Verständnis, manuelles Geschick]","[psychische Gesundheit, physische Gesundheit, Kontaktfreudigkeit, gute Auffassungsgabe, Teamfähigkeit, Kommunikationsfähigkeit, praktisches Verständnis, manuelles Geschick]","[psychische Gesundheit, physische Gesundheit, Kontaktfreudigkeit, gute Auffassungsgabe, Teamfreudigkeit, Kommunikations-Fä

In [3]:
# results_bloom['num_skills_all'] = results_bloom.apply(lambda row: len(row['input_skills_all']))
# results_bloom['num_skills_C'] = results_bloom.apply(lambda row: len(row['input_skills_C']))
results_bloom['num_skills_all'] = results_bloom.apply(lambda row: len(row['expected_skills_all']) if isinstance(row['expected_skills_all'], list) else 0, axis=1)
results_bloom['num_skills_C'] = results_bloom.apply(lambda row: len(row['expected_skills_C']) if isinstance(row['expected_skills_C'], list) else 0, axis=1)

results_bloom['num_gen_skills_all'] = results_bloom.apply(lambda row: len(row['generated_skills_all']) if isinstance(row['generated_skills_all'], list) else 0, axis=1)
results_bloom['num_gen_skills_C'] = results_bloom.apply(lambda row: len(row['generated_skills_C']) if isinstance(row['generated_skills_C'], list) else 0, axis=1)


results_bloom.head(3)

,orig_idx,input,expected_text,generated_text,Problem type,valid,input_skills_C,input_skills_all,expected_skills_all,generated_skills_all,expected_skills_C,generated_skills_C,num_skills_all,num_skills_C,num_gen_skills_all,num_gen_skills_C
0,752,"Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill_C>dienstleitungsorientierten</SoftSkill_C>, <SoftSkill_C>verantwortungsbewussten</SoftSkill_C> wie <SoftSkill>teamorientierten handeln</SoftSkill>.","Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill>dienstleitungsorientierten handeln</SoftSkill>, <SoftSkill>verantwortungsbewussten handeln</SoftSkill> wie <SoftSkill>teamorientierten handeln</SoftSkill>.","Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill>dienstleitungsorientierten handeln</SoftSkill>, <SoftSkill>verantwortungsbewussten handeln</SoftSkill> wie <SoftSkill>teamorientierten handeln</SoftSkill>.",MC,1.0,"[dienstleitungsorientierten, verantwortungsbewussten]","[dienstleitungsorientierten, verantwortungsbewussten, teamorientierten handeln]","[dienstleitungsorientierten handeln, verantwortungsbewussten handeln, teamorientierten handeln]","[dienstleitungsorientierten handeln, verantwortungsbewussten handeln, teamorientierten handeln]","[dienstleitungsorientierten handeln, verantwortungsbewussten handeln]","[dienstleitungsorientierten handeln, verantwortungsbewussten handeln]",3,2,3,2
1,765,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill_C>hoher Dienstleistungs-</SoftSkill_C> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill>hoher Dienstleistungsbereitschaft</SoftSkill> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill>hoher Dienstleistungsbereitschaft</SoftSkill> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,HOWA,1.0,[hoher Dienstleistungs-],"[Arbeitsweise ist proaktiv, hoher Dienstleistungs-, Einsatzbereitschaft]","[Arbeitsweise ist proaktiv, hoher Dienstleistungsbereitschaft, Einsatzbereitschaft]","[Arbeitsweise ist proaktiv, hoher Dienstleistungsbereitschaft, Einsatzbereitschaft]",[hoher Dienstleistungsbereitschaft],[hoher Dienstleistungsbereitschaft],3,1,3,1
2,1655,"Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill_C>psychische</SoftSkill_C> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill_C>Team-</SoftSkill_C> und <SoftSkill>Kommunikations-Fähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.","Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill>psychische Gesundheit</SoftSkill> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill>Teamfähigkeit</SoftSkill> und <SoftSkill>Kommunikationsfähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.","Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill>psychische Gesundheit</SoftSkill> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill>Teamfreudigkeit</SoftSkill> und <SoftSkill>Kommunikations-Fähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.",HMDC,1.0,"[psychische, Team-]","[psychische, physische Gesundheit, Kontaktfreudigkeit, gute Auffassungsgabe, Team-, Kommunikations-Fähigkeit, praktisches Verständnis, manuelles Geschick]","[psychische Gesundheit, physische Gesundheit, Kontaktfreudigkeit, gute Auffassungsgabe, Teamfähigkeit, Kommunikationsfähigkeit, praktisches Verständnis, manuelles Geschick]","[psychische Gesundheit, physische Gesundhe

In [153]:
# results_bloom.loc[results_bloom['num_skills_C'] == 0, 'input'] = results_bloom.loc[results_bloom['num_skills_C'] == 0, 'input'].apply(lambda x: x + '>')

In [4]:
results_bloom[results_bloom['num_skills_C']==0]

,orig_idx,input,expected_text,generated_text,Problem type,valid,input_skills_C,input_skills_all,expected_skills_all,generated_skills_all,expected_skills_C,generated_skills_C,num_skills_all,num_skills_C,num_gen_skills_all,num_gen_skills_C


In [5]:

from collections import Counter

# Create a new column 'skills_match_score'
results_bloom['percent_skills_match_all'] = 0.0
results_bloom['percent_skills_match_C'] = 0.0

results_bloom['num_skills_match_all'] = 0.0
results_bloom['num_skills_match_C'] = 0.0

# Iterate over the rows of the DataFrame
for index, row in results_bloom.iterrows():
    expected_skills_all = set(row['expected_skills_all'])
    generated_skills_all = set(row['generated_skills_all'])
    
    expected_skills_C = set(row['expected_skills_C'])
    generated_skills_C = set(row['generated_skills_C'])

    count_gen_C = Counter(generated_skills_C)
    count_exp_C = Counter(expected_skills_C)

    # Find the common elements between list1 and list2
    common_elements_C = set(expected_skills_C).intersection(set(generated_skills_C))

    # Find the number of common elements, considering duplicates
    num_common_elements_C = sum(min(count_gen_C[element], count_exp_C[element]) for element in common_elements_C)

    count_gen_all = Counter(generated_skills_all)
    count_exp_all = Counter(expected_skills_all)

    # Find the common elements between list1 and list2
    common_elements_all = set(expected_skills_all).intersection(set(generated_skills_all))

    # Find the number of common elements, considering duplicates
    num_common_elements_all = sum(min(count_gen_all[element], count_exp_all[element]) for element in common_elements_all)



    # Calculate the percentage of matching skills
    if len(expected_skills_all) > 0:
        match_score_all = len(expected_skills_all.intersection(generated_skills_all)) / len(expected_skills_all) * 100
    else:
        match_score_all = 0.0
    # Calculate the percentage of matching skills
    if len(expected_skills_C) > 0:
        match_score_C = len(expected_skills_C.intersection(generated_skills_C)) / len(expected_skills_C) * 100
    else:
        match_score_C = 0.0
    
    # Assign the match score to the 'skills_match_score' column
    results_bloom.at[index, 'percent_skills_match_all'] = match_score_all
    results_bloom.at[index, 'percent_skills_match_C'] = match_score_C
    results_bloom.at[index, 'num_skills_match_all'] = num_common_elements_all
    results_bloom.at[index, 'num_skills_match_C'] = num_common_elements_C

In [6]:
results_bloom['Problem type'].value_counts()

Problem type
OWA     63
MWA     53
MC      34
HOWA    19
MDC     11
HMDC    10
NONE     3
HMC      1
Name: count, dtype: int64

In [7]:
results_bloom[results_bloom['percent_skills_match_all'] == 100]['Problem type'].value_counts()

Problem type
OWA     52
MWA     40
MC      27
HOWA    12
MDC      8
NONE     3
HMDC     2
Name: count, dtype: int64

In [8]:
results_bloom[results_bloom['percent_skills_match_C'] == 100]['Problem type'].value_counts()

Problem type
OWA     53
MWA     38
MC      27
HOWA    17
MDC      8
HMDC     3
NONE     3
Name: count, dtype: int64

In [9]:
import Levenshtein
import re 
def remove_sp_char(sent):
    modified_text = re.sub(r'(<)?(?:\/?(?:SoftSkill(?:_C)?))?(>)', '', sent)
    modified_text = re.sub(r'\n', ' ', modified_text)
    modified_text = re.sub(r'\r', ' ', modified_text)
    return modified_text

results_bloom['cleaned_generated_text'] = results_bloom['generated_text'].apply(remove_sp_char)
results_bloom['cleaned_expected_text'] = results_bloom['expected_text'].apply(remove_sp_char)
results_bloom['sentence_levenshtein'] = results_bloom.apply(lambda row: Levenshtein.distance(row['cleaned_generated_text'], row['cleaned_expected_text']), axis=1)
# results_bloom['skills_levenshtein'] = results_bloom.apply(lambda row: Levenshtein.distance(str(row['generated_skills']), str(row['expected_skills'])), axis=1)

In [10]:
results_bloom[results_bloom['orig_idx']==29]

,orig_idx,input,expected_text,generated_text,Problem type,valid,input_skills_C,input_skills_all,expected_skills_all,generated_skills_all,...,num_skills_C,num_gen_skills_all,num_gen_skills_C,percent_skills_match_all,percent_skills_match_C,num_skills_match_all,num_skills_match_C,cleaned_generated_text,cleaned_expected_text,sentence_levenshtein
106,29,<SoftSkill_C>Selbständige</SoftSkill_C> und <SoftSkill>zuverlässige Arbeitsweise</SoftSkill,<SoftSkill>Selbständige Arbeitsweise</SoftSkill> und <SoftSkill>zuverlässige Arbeitsweise</SoftSkill>,<SoftSkill>Selbständige Arbeitsweise</SoftSkill> und <SoftSkill>zuverlässige Arbeitsweise</SoftSkill>\r\n<SoftSkill>Teamfähigkeit</SoftSkill>\r\n<SoftSkill>Kommunikationsfähigkeit</SoftSkill> sowie <SoftSkill>Bereitschaft zur Schichtarbeitung</SoftSkill>\r\n<SoftSkill>Gute Umgangsformen</SoftSkill>\r\n<SoftSkill>Hohes Qualitätsbewusstsein</SoftSkill> und <SoftSkill>Verantwortungsbewusstsein</SoftSkill>,OWA,1.0,[Selbständige],"[Selbständige, zuverlässige Arbeitsweise]","[Selbständige Arbeitsweise, zuverlässige Arbeitsweise]","[Selbständige Arbeitsweise, zuverlässige Arbeitsweise, Teamfähigkeit, Kommunikationsfähigkeit, Bereitschaft zur Schichtarbeitung, Gute Umgangsformen, Hohes Qualitätsbewusstsein, Verantwortungsbewusstsein]",...,1,8,1,100.0,100.0,2.0,1.0,Selbständige Arbeitsweise und zuverlässige Arbeitsweise Teamfähigkeit Kommunikationsfähigkeit sowie Bereitschaft zur Schichtarbeitung Gute Umgangsformen Hohes Qualitätsbewusstsein und Verantwortungsbewusstsein,Selbständige Arbeitsweise und zuverlässige Arbeitsweise,158


In [11]:
from rouge import Rouge
# Initialize the ROUGE scorer

rouge = Rouge()
rouge_score = rouge.get_scores("['wertschätzend', 'respektvoll miteinader kommunizieren', 'konstruktiv im Interesse']", 
                                "['wertschätzend', 'respektvoll miteinader zu kommunizieren', 'konstruktiv im Interesse']")[0]
rouge_score



{'rouge-1': {'r': 0.875, 'p': 1.0, 'f': 0.9333333283555556},
 'rouge-2': {'r': 0.7142857142857143,
  'p': 0.8333333333333334,
  'f': 0.7692307642603551},
 'rouge-l': {'r': 0.875, 'p': 1.0, 'f': 0.9333333283555556}}

In [12]:
# Define functions to calculate Rouge and BLEU scores
def calculate_rouge_scores(expected_list, generated_list):
    rouge = Rouge()
    rouge_scores = []
    min_length = min(len(expected_list), len(generated_list))
    for i in range(min_length):
        expected = expected_list[i]
        generated = generated_list[i]
        scores = rouge.get_scores(expected, generated)
        rouge_scores.append(scores)
    return rouge_scores

def calculate_bleu_score(expected_list, generated_list):
    smoothing = SmoothingFunction().method0
    bleu_scores = []
    for expected, generated in zip(expected_list, generated_list):
        if len(expected_list) == len(generated_list):
            bleu_score = sentence_bleu([expected], generated, smoothing_function=smoothing)
            bleu_scores.append(bleu_score)
    return bleu_scores

def calculate_levenshtein_distance(expected_list, generated_list):
    levenshtein_distances = []
    for expected, generated in zip(expected_list, generated_list):
        if len(expected_list) == len(generated_list):
            distance = Levenshtein.distance(expected, generated)
            levenshtein_distances.append(distance)
    return levenshtein_distances

# Calculate the Rouge and BLEU scores for each row and store them in new columns
results_bloom['rouge_scores_skill_C'] = results_bloom.apply(lambda row: calculate_rouge_scores(row['expected_skills_C'], row['generated_skills_C']), axis=1)
results_bloom['bleu_scores_skill_C'] = results_bloom.apply(lambda row: calculate_bleu_score(row['expected_skills_C'], row['generated_skills_C']), axis=1)

# Calculate the average scores for each pair individually
results_bloom['rouge1_avg_skill_C'] = results_bloom['rouge_scores_skill_C'].apply(lambda x: sum(score[0]['rouge-1']['f'] for score in x) / len(x) if len(x) > 0 else-1)
results_bloom['rouge2_avg_skill_C'] = results_bloom['rouge_scores_skill_C'].apply(lambda x: sum(score[0]['rouge-2']['f'] for score in x) / len(x) if len(x) > 0 else -1)
results_bloom['rougeL_avg_skill_C'] = results_bloom['rouge_scores_skill_C'].apply(lambda x: sum(score[0]['rouge-l']['f'] for score in x) / len(x) if len(x) > 0 else -1)
results_bloom['bleu_avg_skill_C'] = results_bloom['bleu_scores_skill_C'].apply(lambda x: sum(x) / len(x) if len(x) > 0 else -1)

# Calculate the Levenshtein distances for each pair individually and store the sum in a new column
results_bloom['levenshtein_skill_C'] = results_bloom.apply(lambda row: sum(calculate_levenshtein_distance(row['expected_skills_C'], row['generated_skills_C'])), axis=1)


# Calculate the Rouge and BLEU scores for each row and store them in new columns
results_bloom['rouge_scores_skill_all'] = results_bloom.apply(lambda row: calculate_rouge_scores(row['expected_skills_all'], row['generated_skills_all']), axis=1)
results_bloom['bleu_scores_skill_all'] = results_bloom.apply(lambda row: calculate_bleu_score(row['expected_skills_all'], row['generated_skills_all']), axis=1)

# Calculate the average scores for each pair individually
results_bloom['rouge1_avg_skill_all'] = results_bloom['rouge_scores_skill_all'].apply(lambda x: sum(score[0]['rouge-1']['f'] for score in x) / len(x) if len(x) > 0 else -1)
results_bloom['rouge2_avg_skill_all'] = results_bloom['rouge_scores_skill_all'].apply(lambda x: sum(score[0]['rouge-2']['f'] for score in x) / len(x) if len(x) > 0 else -1)
results_bloom['rougeL_avg_skill_all'] = results_bloom['rouge_scores_skill_all'].apply(lambda x: sum(score[0]['rouge-l']['f'] for score in x) / len(x) if len(x) > 0 else -1)
results_bloom['bleu_avg_skill_all'] = results_bloom['bleu_scores_skill_all'].apply(lambda x: sum(x) / len(x) if len(x) > 0 else -1)

# Calculate the Levenshtein distances for each pair individually and store the sum in a new column
results_bloom['levenshtein_skill_all'] = results_bloom.apply(lambda row: sum(calculate_levenshtein_distance(row['expected_skills_all'], row['generated_skills_all'])), axis=1)

results_bloom[results_bloom['orig_idx']==694]



/home/user/ksharma/miniconda3/envs/ks_tars_thesis/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/user/ksharma/miniconda3/envs/ks_tars_thesis/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/user/ksharma/miniconda3/envs/ks_tars_thesis/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram o

,orig_idx,input,expected_text,generated_text,Problem type,valid,input_skills_C,input_skills_all,expected_skills_all,generated_skills_all,...,rougeL_avg_skill_C,bleu_avg_skill_C,levenshtein_skill_C,rouge_scores_skill_all,bleu_scores_skill_all,rouge1_avg_skill_all,rouge2_avg_skill_all,rougeL_avg_skill_all,bleu_avg_skill_all,levenshtein_skill_all
5,694,"Mit Ihrer <SoftSkill_C>offenen</SoftSkill_C> und <SoftSkill>positiven Art</SoftSkill> bedienen Sie unsere Gäste freundlich, <SoftSkill_C>schnell</SoftSkill_C> und nach STARBUCKS® Standards.","Mit Ihrer <SoftSkill>offenen Art</SoftSkill> und <SoftSkill>positiven Art</SoftSkill> bedienen Sie unsere Gäste freundlich, <SoftSkill>bedienen Sie unsere Gäste schnell</SoftSkill> und nach STARBUCKS® Standards.","Mit Ihrer <SoftSkill>offenen Art</SoftSkill> und <SoftSkill>positiven Art</SoftSkill> bedienen Sie unsere Gäste freundlich, <SoftSkill>schnell</SoftSkill> und nach STARBUCKS® Standards.",MDC,1.0,"[offenen, schnell]","[offenen, positiven Art, schnell]","[offenen Art, positiven Art, bedienen Sie unsere Gäste schnell]","[offenen Art, positiven Art, schnell]",...,0.666667,0.512186,26,"[[{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}], [{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}], [{'rouge-1': {'r': 1.0, 'p': 0.2, 'f': 0.33333333055555564}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 1.0, 'p': 0.2, 'f': 0.33333333055555564}}]]","[1.0, 1.0, 0.0243728440732796]",0.777778,0.666667,0.777778,0.674791,26


In [144]:
results_bloom.columns

Index(['orig_idx', 'input', 'expected_text', 'generated_text', 'Problem type',
       'valid', 'input_skills_C', 'input_skills_all', 'expected_skills_all',
       'generated_skills_all', 'expected_skills_C', 'generated_skills_C',
       'num_skills_all', 'num_skills_C', 'num_gen_skills_all',
       'num_gen_skills_C', 'percent_skills_match_all',
       'percent_skills_match_C', 'num_skills_match_all', 'num_skills_match_C',
       'cleaned_generated_text', 'cleaned_expected_text',
       'sentence_levenshtein', 'rouge_scores_skill_C', 'bleu_scores_skill_C',
       'rouge1_avg_skill_C', 'rouge2_avg_skill_C', 'rougeL_avg_skill_C',
       'bleu_avg_skill_C', 'levenshtein_skill_C', 'rouge_scores_skill_all',
       'bleu_scores_skill_all', 'rouge1_avg_skill_all', 'rouge2_avg_skill_all',
       'rougeL_avg_skill_all', 'bleu_avg_skill_all', 'levenshtein_skill_all'],
      dtype='object')

In [164]:
# results_bloom.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/bloom-7b1-lora-ORIG-stopseq20230717114608.csv")


In [13]:
results_bloom['Problem type'].value_counts()

Problem type
OWA     63
MWA     53
MC      34
HOWA    19
MDC     11
HMDC    10
NONE     3
HMC      1
Name: count, dtype: int64

In [166]:
# results_bloom[results_bloom['skills_match_score'] == 100]['Problem type'].value_counts()

In [168]:
# import re 
# def remove_sp_char(sent):
#     modified_text = re.sub(r'(<)?(?:\/?(?:SoftSkill(?:_C)?))?(>)', '', sent)
#     modified_text = re.sub(r'\n', ' ', modified_text)
#     return modified_text

# results_bloom['cleaned_generated_text'] = results_bloom['generated_text'].apply(remove_sp_char)

In [169]:
# results_bloom['input_C'] = results_bloom.apply(lambda row: Levenshtein.distance(row['cleaned_generated_text'], row['cleaned_expected_text']), axis=1)

In [14]:
pd.set_option('display.max_columns', None)  # Display all columns
# results_bloom[results_bloom['num_skills_C'] != results_bloom['num_gen_skills_C']]
results_bloom[results_bloom['num_skills_all'] != results_bloom['num_gen_skills_all']]
results_bloom['mismatched_ALL'] = results_bloom['num_skills_all'] != results_bloom['num_gen_skills_all']
results_bloom[results_bloom['num_skills_all'] != results_bloom['num_gen_skills_all']].head(3)

,orig_idx,input,expected_text,generated_text,Problem type,valid,input_skills_C,input_skills_all,expected_skills_all,generated_skills_all,expected_skills_C,generated_skills_C,num_skills_all,num_skills_C,num_gen_skills_all,num_gen_skills_C,percent_skills_match_all,percent_skills_match_C,num_skills_match_all,num_skills_match_C,cleaned_generated_text,cleaned_expected_text,sentence_levenshtein,rouge_scores_skill_C,bleu_scores_skill_C,rouge1_avg_skill_C,rouge2_avg_skill_C,rougeL_avg_skill_C,bleu_avg_skill_C,levenshtein_skill_C,rouge_scores_skill_all,bleu_scores_skill_all,rouge1_avg_skill_all,rouge2_avg_skill_all,rougeL_avg_skill_all,bleu_avg_skill_all,levenshtein_skill_all,mismatched_ALL
13,1232,eine <SoftSkill_C>gewissenhafte</SoftSkill_C> und <SoftSkill>sehr sorgfältige Arbeitsweise</SoftSkill,eine <SoftSkill>gewissenhafte Arbeitsweise</SoftSkill> und <SoftSkill>sehr sorgfältige Arbeitsweise</SoftSkill>,eine <SoftSkill>gewissenhafte Arbeitsweise</SoftSkill> und <SoftSkill>sehr sorgfältige Arbeitsweise</SoftSkill> sowie <SoftSkill>Kundenorientierung</SoftSkill> und <SoftSkill>Flexibilität</SoftSkill> sind für Sie selbstverständlich.,OWA,1.0,[gewissenhafte],"[gewissenhafte, sehr sorgfältige Arbeitsweise]","[gewissenhafte Arbeitsweise, sehr sorgfältige Arbeitsweise]","[gewissenhafte Arbeitsweise, sehr sorgfältige Arbeitsweise, Kundenorientierung, Flexibilität]",[gewissenhafte Arbeitsweise],[gewissenhafte Arbeitsweise],2,1,4,1,100.000000,100.000000,2.0,1.0,eine gewissenhafte Arbeitsweise und sehr sorgfältige Arbeitsweise sowie Kundenorientierung und Flexibilität sind für Sie selbstverständlich.,eine gewissenhafte Arbeitsweise und sehr sorgfältige Arbeitsweise,75,"[[{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}]]",[1.0],1.000000,1.000000,1.000000,1.000000,0,"[[{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}], [{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}]]",[],1.000000,1.0,1.000000,-1.0,0,True
14,1928,"Technische Grundausbildung im Heizungs-, Sanitär- oder Haustechnikbereich oder Ingenieurstudium\n<SoftSkill>Vielseitiges Interesse an versorgungstechnischen und energiewirtschaftlichen Themen</SoftSkill>\n<SoftSkill>Kundenorientiertes Denken</SoftSkill> und <SoftSkill_C>Handeln</SoftSkill_C>\n<SoftSkill_C>Ausgeprägte Team-</SoftSkill_C>, <SoftSkill_C>Kommunikations-</SoftSkill_C> und <SoftSkill>Organisationsfähigkeit</SoftSkill","Technische Grundausbildung im Heizungs-, Sanitär- oder Haustechnikbereich oder Ingenieurstudium\n<SoftSkill>Vielseitiges Interesse an versorgungstechnischen und energiewirtschaftlichen Themen</SoftSkill>\n<SoftSkill>Kundenorientiertes Denken</SoftSkill> und <SoftSkill>kundenorientiertes Handeln</SoftSkill>\n<SoftSkill>Ausgeprägte Teamfähigkeit</SoftSkill>, <SoftSkill>Kommunikationsfähigkeit</SoftSkill> und <SoftSkill>Organisationsfähigkeit</SoftSkill>","Technische Grundausbildung im Heizungs-, Sanitär- oder Haustechnikbereich oder Ingenieurstudium\n<SoftSkill>Vielseitiges Interesse an versorgungstechnischen und energiewirtschaftlichen Themen</SoftSkill>\n<SoftSkill>Kundenorientiertes Denken</SoftSkill> und <SoftSkill>Kundenorientiertes Handeln</SoftSkill>\n<SoftSkill>Ausgeprägte Teamfähigkeit</SoftSkill>, <SoftSkill>Kommunikationsfähigkeit</SoftSkill> und <SoftSkill>Organisationsfähigkeit</SoftSkill>\n<SoftSkill>Verantwortungsbewusstsein</SoftSkill>, <SoftSkill>Flexibilität</SoftSkill> und <SoftSkill>Exakte Arbeitsweise</SoftSkill>",HMDC,1.0,"[Handeln, Ausgeprägte Team-, Kommunikations-]","[Vielseitiges Interesse an versorgungstechnischen und energiewirtschaftlichen Themen, Kundenorientiertes Denken, Handeln, Ausgeprägte Team-, Kommunikations-, Organisationsfähigkeit]","[Vielseitiges Interesse a

In [15]:
results_C = results_bloom[['orig_idx', 'input', 'expected_text', 'generated_text', 'input_skills_C', 'expected_skills_C', 'generated_skills_C',
                'num_skills_C', 'num_gen_skills_C', 'percent_skills_match_C', 'num_skills_match_C',
                'rouge_scores_skill_C', 'bleu_scores_skill_C', 'rouge1_avg_skill_C',
                'rouge2_avg_skill_C', 'rougeL_avg_skill_C', 'bleu_avg_skill_C', 'levenshtein_skill_C', 'mismatched_ALL','Problem type',
                'valid']]

# Create "results_ALL" DataFrame
results_ALL = results_bloom[['orig_idx', 'input', 'expected_text', 'generated_text', 'input_skills_all', 'expected_skills_all', 'generated_skills_all',
                  'num_skills_all', 'num_gen_skills_all', 'percent_skills_match_all', 'num_skills_match_all',
                  'rouge_scores_skill_all', 'bleu_scores_skill_all', 'rouge1_avg_skill_all',
                  'rouge2_avg_skill_all', 'rougeL_avg_skill_all', 'bleu_avg_skill_all', 'levenshtein_skill_all', 'mismatched_ALL','Problem type',
                'valid']]

# Create "results_SENTENCE" DataFrame
results_SENTENCE = results_bloom[['orig_idx', 'input', 'expected_text', 'generated_text', 'cleaned_generated_text', 'cleaned_expected_text', 'sentence_levenshtein','Problem type',
                'valid']]

# # Export the three DataFrames to separate CSV files (optional)
# results_C.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/BLOOM_results_C.csv", index=False)
# results_ALL.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/BLOOM_results_ALL.csv", index=False)
# results_SENTENCE.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/BLOOM_results_SENTENCE.csv", index=False)

In [16]:
results_ALL[results_ALL['mismatched_ALL']==True].head(5)

,orig_idx,input,expected_text,generated_text,input_skills_all,expected_skills_all,generated_skills_all,num_skills_all,num_gen_skills_all,percent_skills_match_all,num_skills_match_all,rouge_scores_skill_all,bleu_scores_skill_all,rouge1_avg_skill_all,rouge2_avg_skill_all,rougeL_avg_skill_all,bleu_avg_skill_all,levenshtein_skill_all,mismatched_ALL,Problem type,valid
13,1232,eine <SoftSkill_C>gewissenhafte</SoftSkill_C> und <SoftSkill>sehr sorgfältige Arbeitsweise</SoftSkill,eine <SoftSkill>gewissenhafte Arbeitsweise</SoftSkill> und <SoftSkill>sehr sorgfältige Arbeitsweise</SoftSkill>,eine <SoftSkill>gewissenhafte Arbeitsweise</SoftSkill> und <SoftSkill>sehr sorgfältige Arbeitsweise</SoftSkill> sowie <SoftSkill>Kundenorientierung</SoftSkill> und <SoftSkill>Flexibilität</SoftSkill> sind für Sie selbstverständlich.,"[gewissenhafte, sehr sorgfältige Arbeitsweise]","[gewissenhafte Arbeitsweise, sehr sorgfältige Arbeitsweise]","[gewissenhafte Arbeitsweise, sehr sorgfältige Arbeitsweise, Kundenorientierung, Flexibilität]",2,4,100.000000,2.0,"[[{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}], [{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}]]",[],1.000000,1.000000,1.000000,-1.0,0,True,OWA,1.0
14,1928,"Technische Grundausbildung im Heizungs-, Sanitär- oder Haustechnikbereich oder Ingenieurstudium\n<SoftSkill>Vielseitiges Interesse an versorgungstechnischen und energiewirtschaftlichen Themen</SoftSkill>\n<SoftSkill>Kundenorientiertes Denken</SoftSkill> und <SoftSkill_C>Handeln</SoftSkill_C>\n<SoftSkill_C>Ausgeprägte Team-</SoftSkill_C>, <SoftSkill_C>Kommunikations-</SoftSkill_C> und <SoftSkill>Organisationsfähigkeit</SoftSkill","Technische Grundausbildung im Heizungs-, Sanitär- oder Haustechnikbereich oder Ingenieurstudium\n<SoftSkill>Vielseitiges Interesse an versorgungstechnischen und energiewirtschaftlichen Themen</SoftSkill>\n<SoftSkill>Kundenorientiertes Denken</SoftSkill> und <SoftSkill>kundenorientiertes Handeln</SoftSkill>\n<SoftSkill>Ausgeprägte Teamfähigkeit</SoftSkill>, <SoftSkill>Kommunikationsfähigkeit</SoftSkill> und <SoftSkill>Organisationsfähigkeit</SoftSkill>","Technische Grundausbildung im Heizungs-, Sanitär- oder Haustechnikbereich oder Ingenieurstudium\n<SoftSkill>Vielseitiges Interesse an versorgungstechnischen und energiewirtschaftlichen Themen</SoftSkill>\n<SoftSkill>Kundenorientiertes Denken</SoftSkill> und <SoftSkill>Kundenorientiertes Handeln</SoftSkill>\n<SoftSkill>Ausgeprägte Teamfähigkeit</SoftSkill>, <SoftSkill>Kommunikationsfähigkeit</SoftSkill> und <SoftSkill>Organisationsfähigkeit</SoftSkill>\n<SoftSkill>Verantwortungsbewusstsein</SoftSkill>, <SoftSkill>Flexibilität</SoftSkill> und <SoftSkill>Exakte Arbeitsweise</SoftSkill>","[Vielseitiges Interesse an versorgungstechnischen und energiewirtschaftlichen Themen, Kundenorientiertes Denken, Handeln, Ausgeprägte Team-, Kommunikations-, Organisationsfähigkeit]","[Vielseitiges Interesse an versorgungstechnischen und energiewirtschaftlichen Themen, Kundenorientiertes Denken, kundenorientiertes Handeln, Ausgeprägte Teamfähigkeit, Kommunikationsfähigkeit, Organisationsfähigkeit]","[Vielseitiges Interesse an versorgungstechnischen und energiewirtschaftlichen Themen, Kundenorientiertes Denken, Kundenorientiertes Handeln, Ausgeprägte Teamfähigkeit, Kommunikationsfähigkeit, Organisationsfähigkeit, Verantwortungsbewusstsein, Flexibilität, Exakte Arbeitsweise]",6,9,83.333333,5.0,"[[{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}], [{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}], [{'rouge-1': {'r': 0.5, 'p': 0.5, 'f': 0.4999999950000001}, 'rouge-2': {'r':

In [17]:
pd.set_option('display.max_row', None)
results_C['levenshtein_skill_C'].mean()

4.097435897435897

## Sentence ROUGE

In [18]:
from rouge import Rouge

# Function to calculate Rouge scores
def calculate_rouge_scores(reference, candidate):
    rouge = Rouge()
    scores = rouge.get_scores(reference, candidate)
    rouge_1_f = scores[0]['rouge-1']['f'] if 'rouge-1' in scores[0] else None
    rouge_2_f = scores[0]['rouge-2']['f'] if 'rouge-2' in scores[0] else None
    rouge_l_f = scores[0]['rouge-l']['f'] if 'rouge-l' in scores[0] else None
    return rouge_1_f, rouge_2_f, rouge_l_f

# Apply the functions to calculate the scores and add them as new columns
results_SENTENCE['rouge1_score'], results_SENTENCE['rouge2_score'], results_SENTENCE['rougeL_score'] = zip(*results_SENTENCE.apply(
    lambda row: calculate_rouge_scores(row['cleaned_expected_text'], row['cleaned_generated_text']),
    axis=1
))

/tmp/ipykernel_579343/809110196.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_SENTENCE['rouge1_score'], results_SENTENCE['rouge2_score'], results_SENTENCE['rougeL_score'] = zip(*results_SENTENCE.apply(
/tmp/ipykernel_579343/809110196.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_SENTENCE['rouge1_score'], results_SENTENCE['rouge2_score'], results_SENTENCE['rougeL_score'] = zip(*results_SENTENCE.apply(
/tmp/ipykernel_579343/809110196.py:13: SettingWithCopyWarning: 
A value is try

## Sentence BERT Cosine Similarity

In [19]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch

# Assuming you have already read the DataFrame from the CSV file
# results_SENTENCE = pd.read_csv("/path/to/results_SENTENCE.csv")

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to calculate cosine similarity
def calculate_cosine_similarity(sentence1, sentence2):
    # Tokenize and convert sentences to tensors
    inputs = tokenizer([sentence1, sentence2], return_tensors="pt", padding=True, truncation=True)

    # Get BERT embeddings
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :].numpy()

    # Normalize embeddings
    normalized_embeddings = embeddings / (embeddings ** 2).sum(axis=-1, keepdims=True) ** 0.5

    # Calculate cosine similarity
    similarity = (normalized_embeddings[0] @ normalized_embeddings[1].T).item()
    return similarity

# Apply the function to calculate cosine similarity and add it as a new column
results_SENTENCE['cosine_similarity'] = results_SENTENCE.apply(lambda row: calculate_cosine_similarity(row['cleaned_generated_text'], row['cleaned_expected_text']), axis=1)


/tmp/ipykernel_579343/2919956664.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_SENTENCE['cosine_similarity'] = results_SENTENCE.apply(lambda row: calculate_cosine_similarity(row['cleaned_generated_text'], row['cleaned_expected_text']), axis=1)


## FUnction for cosine similarity for _C and ALL

In [20]:

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to get BERT embeddings for a list of sentences
def get_bert_embeddings(sentences):
    inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :].numpy()
    return embeddings

# Function to calculate cosine similarity between two lists of sentences
def calculate_cosine_similarity(list1, list2):
    if not list1 or not list2:
        return np.nan

    # Get BERT embeddings for both lists of sentences
    embeddings1 = get_bert_embeddings(list1)
    embeddings2 = get_bert_embeddings(list2)

    # Normalize embeddings
    normalized_embeddings1 = embeddings1 / (embeddings1 ** 2).sum(axis=-1, keepdims=True) ** 0.5
    normalized_embeddings2 = embeddings2 / (embeddings2 ** 2).sum(axis=-1, keepdims=True) ** 0.5

    # Calculate cosine similarity for each sentence pair
    similarity_values = np.dot(normalized_embeddings1, normalized_embeddings2.T)
    return similarity_values.diagonal().tolist()


In [22]:

import numpy as np
# Apply the function to calculate pairwise cosine similarity and store the distances in a list
cosine_similarity_list = []
for index, row in results_C.iterrows():
    list1 = row['expected_skills_C']
    list2 = row['generated_skills_C']

    # Calculate cosine similarity for each sentence pair in both lists
    pairwise_similarities = calculate_cosine_similarity(list1, list2)

    # Append the list of pairwise similarities for the row
    cosine_similarity_list.append(pairwise_similarities)

# Add the list of pairwise cosine similarity distances as a new column in the DataFrame
results_C['pairwise_cosine_similarity'] = cosine_similarity_list



# Apply the function to calculate pairwise cosine similarity and store the distances in a list
cosine_similarity_list = []
for index, row in results_ALL.iterrows():
    list1 = row['expected_skills_all']
    list2 = row['generated_skills_all']

    # Calculate cosine similarity for each sentence pair in both lists
    pairwise_similarities = calculate_cosine_similarity(list1, list2)

    # Append the list of pairwise similarities for the row
    cosine_similarity_list.append(pairwise_similarities)

# Add the list of pairwise cosine similarity distances as a new column in the DataFrame
results_ALL['pairwise_cosine_similarity'] = cosine_similarity_list

results_C['average_cosine_similarity'] = results_C['pairwise_cosine_similarity'].apply(lambda x: np.mean(x) if x else np.nan)
results_ALL['average_cosine_similarity'] = results_ALL['pairwise_cosine_similarity'].apply(lambda x: np.mean(x) if x else np.nan)


/tmp/ipykernel_579343/2056329418.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_C['pairwise_cosine_similarity'] = cosine_similarity_list
/tmp/ipykernel_579343/2056329418.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_ALL['pairwise_cosine_similarity'] = cosine_similarity_list
/tmp/ipykernel_579343/2056329418.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [157]:
# results_ALL[results_ALL['mismatched_ALL']==True].head(5)

In [23]:
results_SENTENCE.columns

Index(['orig_idx', 'input', 'expected_text', 'generated_text',
       'cleaned_generated_text', 'cleaned_expected_text',
       'sentence_levenshtein', 'Problem type', 'valid', 'rouge1_score',
       'rouge2_score', 'rougeL_score', 'cosine_similarity'],
      dtype='object')

## Prepare Metrics and Tables 

In [24]:
# Create a DataFrame for _C metrics
summary_C_df = pd.DataFrame({
    'Metric': ['Average Rouge-1 Score (C)', 'Average Rouge-2 Score (C)',
               'Average Rouge-L Score (C)',  'Average Levenshtein (C)',
               'Weighted Avg. Percentage Skills (C)', 'Cosine Similarity(C)'],
    'Value': [results_C['rouge1_avg_skill_C'].mean(), results_C['rouge2_avg_skill_C'].mean(),
              results_C['rougeL_avg_skill_C'].mean(),results_C['levenshtein_skill_C'].mean(),
              results_C['num_skills_match_C'].sum() / results_C['num_skills_C'].sum(),results_C['average_cosine_similarity'].mean()]
})

# Create a DataFrame for "ALL" metrics
summary_ALL_df = pd.DataFrame({
    'Metric': ['Average Rouge-1 Score (ALL)', 'Average Rouge-2 Score (ALL)',
               'Average Rouge-L Score (ALL)', 'Average Levenshtein (ALL)',
               'Weighted Avg. Percentage Skills (ALL)', 'Cosine Similarity(ALL)'],
    'Value': [results_ALL['rouge1_avg_skill_all'].mean(), results_ALL['rouge2_avg_skill_all'].mean(),
              results_ALL['rougeL_avg_skill_all'].mean(), results_ALL['levenshtein_skill_all'].mean(),
              results_ALL['num_skills_match_all'].sum() / results_ALL['num_skills_all'].sum(),results_ALL['average_cosine_similarity'].mean()]
})

# Create a DataFrame for "ALL" metrics
summary_SENTENCE_df = pd.DataFrame({
    'Metric': ['Average Rouge-1 Score (SENTENCE)', 'Average Rouge-2 Score (SENTENCE)',
               'Average Rouge-L Score (SENTENCE)', 'Average Levenshtein (SENTENCE)',
               'Cosine Similarity(SENTENCE)'],
    'Value': [results_SENTENCE['rouge1_score'].mean(), results_SENTENCE['rouge2_score'].mean(),
              results_SENTENCE['rougeL_score'].mean(), results_SENTENCE['sentence_levenshtein'].mean(),
              results_SENTENCE['cosine_similarity'].mean()]
})

# Export the summary DataFrames to CSV
# summary_C_df.to_csv("summary_metrics_C.csv", index=False)
# summary_ALL_df.to_csv("summary_metrics_ALL.csv", index=False)

In [25]:
summary_C_df

,Metric,Value
0,Average Rouge-1 Score (C),0.915904
1,Average Rouge-2 Score (C),0.816918
2,Average Rouge-L Score (C),0.915904
3,Average Levenshtein (C),4.097436
4,Weighted Avg. Percentage Skills (C),0.801498
5,Cosine Similarity(C),0.990186


In [26]:
summary_ALL_df

,Metric,Value
0,Average Rouge-1 Score (ALL),0.953285
1,Average Rouge-2 Score (ALL),0.828881
2,Average Rouge-L Score (ALL),0.952919
3,Average Levenshtein (ALL),3.010256
4,Weighted Avg. Percentage Skills (ALL),0.899536
5,Cosine Similarity(ALL),0.994917


In [27]:
summary_SENTENCE_df

,Metric,Value
0,Average Rouge-1 Score (SENTENCE),0.960309
1,Average Rouge-2 Score (SENTENCE),0.925835
2,Average Rouge-L Score (SENTENCE),0.960309
3,Average Levenshtein (SENTENCE),14.815385
4,Cosine Similarity(SENTENCE),0.983523


## Problem Types DATA RESULTS _C SKILLS

In [28]:
summary_C_df_list = []

# Group the DataFrame by "Problem Type"
grouped_results_C = results_C.groupby('Problem type')

# Iterate over each group and calculate the summary DataFrame
for problem_type, group in grouped_results_C:
    num_samples = len(group)  # Calculate the number of samples for the current category
    summary_C_df = pd.DataFrame({
        'Average Rouge-1 Score (C)': [group['rouge1_avg_skill_C'].mean()],
        'Average Rouge-2 Score (C)': [group['rouge2_avg_skill_C'].mean()],
        'Average Rouge-L Score (C)': [group['rougeL_avg_skill_C'].mean()],
        'Average Levenshtein (C)': [group['levenshtein_skill_C'].mean()],
        'Weighted Avg. Percentage Skills (C)': [group['num_skills_match_C'].sum() / group['num_skills_C'].sum()],
        'Cosine Similarity (C)': [group['average_cosine_similarity'].mean()],
        'Number of Samples': [num_samples]
    }, index=[problem_type])
    # Add the summary DataFrame for the current category to the list
    summary_C_df_list.append(summary_C_df)

# Concatenate the summary DataFrames into a single DataFrame
result_summary_df = pd.concat(summary_C_df_list)

# Calculate overall averages across all categories and add as a new row
overall_averages = {
    'Average Rouge-1 Score (C)': results_C['rouge1_avg_skill_C'].mean(),
    'Average Rouge-2 Score (C)': results_C['rouge2_avg_skill_C'].mean(),
    'Average Rouge-L Score (C)': results_C['rougeL_avg_skill_C'].mean(),
    'Average Levenshtein (C)': results_C['levenshtein_skill_C'].mean(),
    'Weighted Avg. Percentage Skills (C)': results_C['num_skills_match_C'].sum() / results_C['num_skills_C'].sum(),
    'Cosine Similarity (C)': results_C['average_cosine_similarity'].mean(),
    'Number of Samples': len(results_C)
}

overall_summary_df = pd.DataFrame(overall_averages, index=['All Samples'])

# Concatenate the overall summary as a new row to the result_summary_df
result_summary_df_C_skills = pd.concat([result_summary_df, overall_summary_df])

result_summary_df_C_skills['model'] = 'Bloom'
result_summary_df_C_skills


,Average Rouge-1 Score (C),Average Rouge-2 Score (C),Average Rouge-L Score (C),Average Levenshtein (C),Weighted Avg. Percentage Skills (C),Cosine Similarity (C),Number of Samples,model
HMC,0.833333,0.500000,0.833333,5.000000,0.500000,0.965937,1,Bloom
HMDC,0.792976,0.567222,0.792976,15.200000,0.640000,0.975839,10,Bloom
HOWA,0.921053,0.684211,0.921053,0.947368,0.894737,0.996902,19,Bloom
MC,0.921133,0.870798,0.921133,5.705882,0.835616,0.989173,34,Bloom
MDC,0.914141,0.833333,0.914141,5.454545,0.869565,0.981785,11,Bloom
MWA,0.908213,0.830720,0.908213,4.169811,0.714286,0.989208,53,Bloom
NONE,1.000000,0.333333,1.000000,0.000000,1.000000,1.000000,3,Bloom
OWA,0.933790,0.894044,0.933790,2.365079,0.843750,0.993035,63,Bloom
All Samples,0.915904,0.816918,0.915904,4.097436,0.801498,0.990186,195,Bloom


## Problem Types DATA RESULTS ALL SKILLS

In [29]:
# Create an empty list to store the summary DataFrames for each category
summary_ALL_df_list = []

# Group the DataFrame by "Problem Type"
grouped_results_ALL = results_ALL.groupby('Problem type')

# Iterate over each group and calculate the summary DataFrame
for problem_type, group in grouped_results_ALL:
    num_samples = len(group)  # Calculate the number of samples for the current category
    summary_ALL_df = pd.DataFrame({
        'Average Rouge-1 Score (ALL)': [group['rouge1_avg_skill_all'].mean()],
        'Average Rouge-2 Score (ALL)': [group['rouge2_avg_skill_all'].mean()],
        'Average Rouge-L Score (ALL)': [group['rougeL_avg_skill_all'].mean()],
        'Average Levenshtein (ALL)': [group['levenshtein_skill_all'].mean()],
        'Weighted Avg. Percentage Skills (ALL)': [group['num_skills_match_all'].sum() / group['num_skills_all'].sum()],
        'Cosine Similarity (ALL)': [group['average_cosine_similarity'].mean()],
        'Number of Samples': [num_samples]
    }, index=[problem_type])
    # Add the summary DataFrame for the current category to the list
    summary_ALL_df_list.append(summary_ALL_df)

# Concatenate the summary DataFrames into a single DataFrame
result_summary_ALL_df = pd.concat(summary_ALL_df_list)

# Calculate overall averages across all categories and append as a new row
overall_averages_ALL = {
    'Average Rouge-1 Score (ALL)': results_ALL['rouge1_avg_skill_all'].mean(),
    'Average Rouge-2 Score (ALL)': results_ALL['rouge2_avg_skill_all'].mean(),
    'Average Rouge-L Score (ALL)': results_ALL['rougeL_avg_skill_all'].mean(),
    'Average Levenshtein (ALL)': results_ALL['levenshtein_skill_all'].mean(),
    'Weighted Avg. Percentage Skills (ALL)': results_ALL['num_skills_match_all'].sum() / results_ALL['num_skills_all'].sum(),
    'Cosine Similarity (ALL)': results_ALL['average_cosine_similarity'].mean(),
    'Number of Samples': len(results_ALL)
}

overall_summary_ALL_df = pd.DataFrame(overall_averages_ALL, index=['All Samples'])

# Concatenate the overall summary as a new row to the result_summary_ALL_df
result_summary_df_ALL_skills = pd.concat([result_summary_ALL_df, overall_summary_ALL_df])

result_summary_df_ALL_skills['model'] = 'Bloom'
result_summary_df_ALL_skills


,Average Rouge-1 Score (ALL),Average Rouge-2 Score (ALL),Average Rouge-L Score (ALL),Average Levenshtein (ALL),Weighted Avg. Percentage Skills (ALL),Cosine Similarity (ALL),Number of Samples,model
HMC,0.777778,0.333333,0.777778,19.000000,0.333333,0.970903,1,Bloom
HMDC,0.857222,0.620833,0.857222,7.200000,0.745098,0.983990,10,Bloom
HOWA,0.914181,0.487719,0.914181,2.631579,0.873016,0.996216,19,Bloom
MC,0.954847,0.914391,0.954847,4.029412,0.909774,0.994475,34,Bloom
MDC,0.939646,0.860606,0.939646,3.636364,0.914894,0.987595,11,Bloom
MWA,0.963260,0.868411,0.961912,3.301887,0.910828,0.995599,53,Bloom
NONE,1.000000,0.666667,1.000000,0.000000,1.000000,1.000000,3,Bloom
OWA,0.973292,0.903308,0.973292,1.492063,0.934783,0.997262,63,Bloom
All Samples,0.953285,0.828881,0.952919,3.010256,0.899536,0.994917,195,Bloom


## Problem Types DATA RESULTS SENTENCE SKILLS

In [30]:
# Create an empty list to store the summary DataFrames for each category
summary_SENTENCE_df_list = []

# Group the DataFrame by "Problem Type"
grouped_results_SENTENCE = results_SENTENCE.groupby('Problem type')

# Iterate over each group and calculate the summary DataFrame
for problem_type, group in grouped_results_SENTENCE:
    num_samples = len(group)  # Calculate the number of samples for the current category
    summary_SENTENCE_df = pd.DataFrame({
        'Average Rouge-1 Score (SENTENCE)': [group['rouge1_score'].mean()],
        'Average Rouge-2 Score (SENTENCE)': [group['rouge2_score'].mean()],
        'Average Rouge-L Score (SENTENCE)': [group['rougeL_score'].mean()],
        'Average Levenshtein (SENTENCE)': [group['sentence_levenshtein'].mean()],
        'Cosine Similarity (SENTENCE)': [group['cosine_similarity'].mean()],
        'Number of Samples': [num_samples]
    }, index=[problem_type])
    # Add the summary DataFrame for the current category to the list
    summary_SENTENCE_df_list.append(summary_SENTENCE_df)

# Concatenate the summary DataFrames into a single DataFrame
result_summary_SENTENCE_df = pd.concat(summary_SENTENCE_df_list)

# Calculate overall averages across all categories and add as a new row
overall_averages_SENTENCE = {
    'Average Rouge-1 Score (SENTENCE)': results_SENTENCE['rouge1_score'].mean(),
    'Average Rouge-2 Score (SENTENCE)': results_SENTENCE['rouge2_score'].mean(),
    'Average Rouge-L Score (SENTENCE)': results_SENTENCE['rougeL_score'].mean(),
    'Average Levenshtein (SENTENCE)': results_SENTENCE['sentence_levenshtein'].mean(),
    'Cosine Similarity (SENTENCE)': results_SENTENCE['cosine_similarity'].mean(),
    'Number of Samples': len(results_SENTENCE)
}

overall_summary_SENTENCE_df = pd.DataFrame(overall_averages_SENTENCE, index=['All Samples'])

# Concatenate the overall summary as a new row to the result_summary_SENTENCE_df
result_summary_df_SENTENCE = pd.concat([result_summary_SENTENCE_df, overall_summary_SENTENCE_df])

result_summary_df_SENTENCE['model'] = 'Bloom'
result_summary_df_SENTENCE

,Average Rouge-1 Score (SENTENCE),Average Rouge-2 Score (SENTENCE),Average Rouge-L Score (SENTENCE),Average Levenshtein (SENTENCE),Cosine Similarity (SENTENCE),Number of Samples,model
HMC,0.933333,0.571429,0.933333,19.000000,0.992181,1,Bloom
HMDC,0.956600,0.861404,0.956600,21.200000,0.991853,10,Bloom
HOWA,0.983570,0.942876,0.983570,3.263158,0.992453,19,Bloom
MC,0.969503,0.943287,0.969503,11.676471,0.990143,34,Bloom
MDC,0.983409,0.934333,0.983409,11.454545,0.991802,11,Bloom
MWA,0.971182,0.933908,0.971182,21.358491,0.982437,53,Bloom
NONE,1.000000,1.000000,1.000000,0.333333,1.000000,3,Bloom
OWA,0.933648,0.914146,0.933648,14.920635,0.974220,63,Bloom
All Samples,0.960309,0.925835,0.960309,14.815385,0.983523,195,Bloom


In [32]:
result_summary_df_C_skills['Problem Type'] = result_summary_df_C_skills.index
result_summary_df_ALL_skills['Problem Type'] = result_summary_df_ALL_skills.index
result_summary_df_SENTENCE['Problem Type'] = result_summary_df_SENTENCE.index
# Reset the index to create a new default integer index
# results_df_filtered.reset_index(drop=True, inplace=True)

In [35]:
result_summary_df_C_skills.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/summary_metrics_C_BLOOM.csv", index=True)
result_summary_df_ALL_skills.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/summary_metrics_ALL_BLOOM.csv", index=True)
result_summary_df_SENTENCE.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/summary_metrics_SENTENCE_BLOOM.csv", index=True)